# Synthea Modules by Progressive Refinement

## Configurations

In [574]:
import os
import google.generativeai as gemini
from anthropic import Anthropic
from openai import OpenAI

# "claude" or "gemini" or "gpt"
generator = "claude"    
reviewer = "claude"
RATE_LIMIT_SLEEP = 5  # seconds to sleep between calls (claude only)

# temperature
temp = 0.75

disease = "hyperthyroidism"
disease_profile_file = "hyperthyroidism_disease_profile_6.txt"
synthea_reference_file = "synthea_reference_manual.txt"
synthea_examples_file = "synthea_module_examples.txt"

claude_api_key = os.getenv('ANTHROPIC_API_KEY2')   # preferred over 'ANTHROPIC_API_KEY'
gemini_api_key = os.getenv('GEMINI_API_KEY')
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

gemini.configure(api_key=gemini_api_key)
gemini_version = "models/gemini-1.5-pro"      # "models/gemini-1.5-flash" "models/gemini-1.5-pro" 
gemini_max_output_tokens = 8192

claude = Anthropic(api_key = claude_api_key)
claude_version = "claude-3-5-sonnet-20240620"  # "claude-3-opus-20240229"   "claude-3-5-sonnet-20240620" "claude-3-sonnet-20240229" "claude-3-haiku-20240307"
claude_max_output_tokens = 8192  # claude 3 opus is only 4096 tokens, sonnet is 8192

gpt = OpenAI()
gpt_version =  "gpt-4o"  # no access to "gpt-4o-64k-output-alpha" 
gpt_max_output_tokens = 4096  # "gpt-4o-64k-output-alpha" is 64000 output tokens max

debug = False

### Utilities

In [575]:
import io, threading, time, re, json
import pandas as pd
import numpy as np
from json_repair import repair_json

def upload_file(file_path):
    try:
        with open(file_path, 'r') as file:
            content = file.read()
        return content
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        return content

synthea_reference = upload_file(synthea_reference_file)
synthea_examples = upload_file(synthea_examples_file)
disease_profile = upload_file(disease_profile_file)

def print_claude_messages(dict_list, max_length=200):
    for item in dict_list:
        role = item['role']
        content = item['content'][:max_length]
        if len(item['content']) > max_length:
            content += '...'
        print(f"Role: {role}, Content: {content}")

def heartbeat(stop_event, start_time):
    """Prints elapsed time every second until told to stop."""
    elapsed_time = 0
    while not stop_event.is_set():
        elapsed_time = int(time.time() - start_time)
        print(f"{elapsed_time} seconds", end="\r", flush=True)
        time.sleep(1)
    # Print final time
    elapsed_time = int(time.time() - start_time)
    print(f"Total time taken: {elapsed_time} seconds")
    
def is_valid_json(text):
    try:
        json.loads(text)
        return True
    except:
        return False

## Prompts

In [576]:
background = """
***Background***
Synthea is a synthetic patient generator that models the medical history of synthetic patients. 
Synthea's mission is to create high-quality synthetic, realistic but not real, patient data and associated health records covering every aspect of healthcare.
A Synthea module is a JSON document primarily consisting of named states and transitions between the states. The types of states include: 
* Control Types: Initial, Terminal, Simple, Guard, Delay, SetAttribute, Counter, CallSubmodule
* Clinical Types: Encounter, EncounterEnd, ConditionOnset, ConditionEnd, AllergyOnset, AllergyEnd, MedicationOrder, MedicationEnd, CarePlanStart, CarePlanEnd, Procedure, ImagingStudy, Device, DeviceEnd, SupplyList. Observation, MultiObservation, DiagnosticReport, Symptom, Death

Synthea modules must strictly follow these rules: 
* IMPORTANT! Every state must be the target of a transition from another state, except for the Initial state. There can be no disconnected states.
* Every state except the terminal state must have an output transition.
* All states much be reachable. This means there must be a directed path from the Initial state to every other state. 
* There must be an age guard or delay directly after the initial state. If the disease never happens until a patient is a certain age or age range, use a Delay. If a disease never happens unless a certain condition is met (e.g., the person acquires a certain other condition), use a Guard. If the disease can happen soon after birth use short delay state (for example 1 day, week, or month).
* Every Encounter must be paired with an EncounterEnd.
* Every clinical action (Procedure, Observation, MedicationOrder, ImagingStudy) must occur between an Encounter and EncounterEnd.
* ConditionOnset, ConditionEnd, and Symptom states should not happen during an Encounter. Typically conditions begin first, symptoms second, and encounters third.  
* In distributed transitions, all weights must add up to 100%.
* The name of the state appears as the primary key for the state (as explained in the synthea_module_examples).

Here are some other important things to note:
* Only a percentage of the population will get the disease. Many people never get the disease. 
* Not all patients who get the disease get all symptoms. In a Symptom state, the "probability" represents the likelihood of experiencing the symptom, while the range represents the severity of the symptom.
* Symptoms should appear in series, not parallel, unless they are truly mutually exclusive.
* Not all patients get all diagnostic tests. Use your knowledge to determine what tests a patient should receive based on their presentation.
* Use the disease profile to determine which patients get which treatments.
* Every Synthea module must begin with a single Initial state. 
* The Terminal state represents the end of the disease course, and the termination of the module. It does not represent death. If patients die, use the Death state.
* Cyclical paths are legal.
* Self-transitions, where the target of the transition is the same state that defines the transition, are legal. A self-transition is used to return to the same state in the next time step and are often used with distributed transitions when there is no progression to the next state.
* Direct transitions transition directly to the indicated state. Distributed transitions transition to one of several possible states based on a probability distribution. For example, a value of 0.55 would indicate a 55% chance of transitioning to the corresponding state. Conditional transitions transition to one of several possible states based on conditional (if-then) logic. A conditional transition consists of an array of condition/transition pairs that are tested in the order they are defined. The first condition that evaluates to true will result in a transition to its corresponding transition state. Conditional logic supports And, Or, Not, At Least, At Most operations upon gender, race, socioeconomic status, and any other patient attribute. Conditional logic can also access the medical record to look at observations, conditions, medications, and care plans.

***Resources***
Please take this opportunity to read and understand the following information:
* <synthea_reference> html tag: gives details of the various Synthea states and how to connect those states using transitions.
* <synthea_module_examples> html tag: gives several examples of Synthea modules for other diseases. These are good models for what we are trying to accomplish.
Make sure you understand each of these resources in detail before going any further.
"""

create_prompt = f"""
***Task***
Your task is to create a Synthea module for {disease}.
The disease_profile file (indicated by the <disease_profile> html tag) contains requirements for modeling {disease} that will serve as your basis for building the module.
You are to think step by step about the entire module before generating your final answer.
Each requirement in the disease profile is numbered so it can be easily cited.

***Rules to Follow***
* Every state and transition in the module you create must relate to one or more requirements in the disease profile.
* Every state must include a complete explanation for why it exists. Put the explanation in the "remarks" field.
* Every state must cite a requirement number or numbers from the disease profile that justify the state. Put the reference number(s) in a new property called "requirement_number".

***Output***
The output will be MINIFIED JSON with all output compressed into a single line, to minimize the size of the output.
The output will be valid Synthea module following all the rules expressed in the background section.
The "requirement_number" field will contain the fact number or numbers implemented by the state, in this format: "requirement_number": "14, 16, 20" 
The "remarks" field will include a complete explanation of the state, its properties, and its transitions.
"""

begin_module_output = '{"name": "' +disease+'","remarks:":'

review_prompt = """
***Role***
You are an expert on Synthea, the synthetic patient data simulator. You have mastered the background material and resources provided.

***Task***
Perform a thorough, thoughtful review of the Synthea module that is provided.
You will provide a detailed explanation of how each requirement is (or is not) represented in the Synthea module.
Your analysis will be encapsulated in a numerical score that reflects whether the requirement is adequately modeled in the module. 

***Analysis Method***
Your review will use the provided disease profile and confirm that all the requirements in the disease profile are reflected in the Synthea module provided.
You will go through each requirement in the disease profile individually, one at a time, in numerical order, and see if that requirement is correctly implemented.
Points to check:
(a) Is there a state or states in the module that implement the requirement?
(b) Are the properties of that state or those states well-formed and reflective of the requirement?
(c) If the requirement describes a sequence of events, do the states before and after connect to each other in the right order?  
(d) Are the input and output transition probabilities and other state parameters consistent with the requirement?

***Output***
The output will be a table with six columns and with one row per requirement:
Column 0 (requirement_number) will have the requirement number from the disease profile (1, 2, 3, etc.)
Column 1 (Requirement) will have the full text of the requirement or disease fact quoted from the disease profile. Do not truncate.
Column 2 (Explanation) will have a complete, detailed explanation of how the requirement is (or is not) represented in the Synthea module.
Column 3 (Transitions) will have a complete, detailed explanation of the transitions before, after, and between the states involved with the requirement support the requirement. 
Column 4 (Change) will contain a detailed description of changes, if any, that are required to implement the requirement, or improve the current implementation. If no change is required, print "none".
Column 5 (Score) will contain a score on how well the requirement has been implemented, as follows: 
    0.0: The requirement has not been implemented at all or the requirement is not applicable (N/A).
    0.25: The requirement has been implemented in some fashion, but the implementation is incorrect. 
    0.5: The requirement has been partly implemented, but does not fully capture the requirement.
    0.75: The requirement has been fully implemented but may have minor problem, such as an inaccurate transition probability.
    1.0: The requirement has been fully and correctly implemented.
The overall score should account for the requirement, explanation, transitions, and suggested change.
"""

continuation_prompt = """
Please continue the output from the point you left off to complete the JSON-format output. 
Make sure the beginning of the continuation matches exactly where the previous answer left off, and when the two parts are concatenated, the result is valid JSON.
"""

update_prompt = """
It looks like you have made some mistakes and/or missed some requirements or just implemented them poorly.

Here is a list of things that can be improved: 

{0}

***Task***
Make the necessary corrections to the Synthea module. Here is the Synthea module you are trying to improve:

<synthea_module>
{1}
</synthea_module>

When making modifications to the module, you will not make unnecessary changes,
You will preserve features that are not involved in the current improvements.

***Output***
The output will be MINIFIED JSON with all output compressed into a single line, to minimize the size of the output.
The output will be valid Synthea module following all the rules expressed in the background section.
The "requirement_number" field will contain the fact number or numbers implemented by the state, in this format: "requirement_number": "14, 16, 20" 
The "remarks" field will include a complete justification and explanation of the state, its properties, and its transitions.

Again, here are the issues you must address: 
{0}
"""

structure_syntax_improve_prompt = """

It looks like there are some mistakes in this Synthea module:

<synthea_module>
{0}
</synthea_module>

Here is a list of errors that you need to fix: 
<errors_to_fix>
{1}
</errors_to_fix>

***Task***
You will go through each error in the list and make changes to the JSON Synthea module that fix the problem.
Each error description has a suggested approach you will follow.

***Output***
The output will be MINIFIED JSON with all output compressed into a single line, to minimize the size of the output.
The output will be valid Synthea module following all the rules expressed in the background section.

To repeat, here are the issues you must address: 
<errors_to_fix>
{1}
</errors_to_fix>
"""


## Functions For Generating Synthea Modules

In [577]:
def message_gpt(prompt, assistant=None, thread=None):
    if assistant is None:
        assistant = gpt.beta.assistants.create(
            name="Synthea Assistant",
            instructions="You are an expert in creating synthetic health data using Synthea, the synthetic health data simulator.",
            model=gpt_version,
            #tools=[{"type": "file_search"}],
            #tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
        )
    if thread is None:
        thread = gpt.beta.threads.create()
    # Attach new message to the thread
    thread_message = gpt.beta.threads.messages.create(thread.id, role="user", content=prompt)
    start_time = time.time()
    stop_event = threading.Event()
    heartbeat_thread = threading.Thread(target=heartbeat, args=(stop_event, start_time))
    heartbeat_thread.start()
    try:
        # run the thread
        run = gpt.beta.threads.runs.create_and_poll(thread_id=thread.id, assistant_id=assistant.id, temperature=temp)
    finally:
        stop_event.set()  # Signal the heartbeat to stop
        heartbeat_thread.join()  # Wait for the heartbeat thread to finish
    if run.status == 'completed': 
        messages = list(gpt.beta.threads.messages.list(thread_id=thread.id))
        first_message = messages[0]  # Access the first message directly
        message_content = first_message.content[0].text.value
        return message_content, assistant, thread
    else:
        print("Run did not complete, status = ", run.status)

# gemini maintains history in a chat session object. Messages are sent to a specific chat session.
def message_gemini(prompt, chat_session=None):
    global gemini_version, temp, gemini_max_output_tokens
    if chat_session is None:
        model = gemini.GenerativeModel(model_name=gemini_version, generation_config={"temperature": temp, "max_output_tokens": gemini_max_output_tokens, "response_mime_type": "application/json"})
        chat_session = model.start_chat()
    start_time = time.time()
    stop_event = threading.Event()
    heartbeat_thread = threading.Thread(target=heartbeat, args=(stop_event, start_time))
    heartbeat_thread.start()
    try:
        result = chat_session.send_message(prompt, stream=False)
    finally:
        stop_event.set()  # Signal the heartbeat to stop
        heartbeat_thread.join()  # Wait for the heartbeat thread to finish
    return result, chat_session

# claude maintains chat history in an array of messages alternating between 'user' and 'assistant'.
# the optional assistant_prompt is the beginning of the output that Claude will extend
def message_claude(user_prompt, assistant_prompt=None, messages=None):
    global claude_version, claude_max_output_tokens, temp, debug
    if messages is None:
        messages = [{"role": "user","content": user_prompt}]
    else:
        messages.append({"role": "user","content": user_prompt})
    if assistant_prompt is not None:
        messages.append({"role": "assistant", "content": assistant_prompt})
    if debug:
        print("About to message_claude with these messages:")
        print_claude_messages(messages)
    start_time = time.time()
    stop_event = threading.Event()
    heartbeat_thread = threading.Thread(target=heartbeat, args=(stop_event, start_time))
    heartbeat_thread.start()
    response_text = None
    try:
        response = claude.messages.create(model=claude_version, max_tokens=claude_max_output_tokens, messages=messages, temperature=temp)
        response_text = response.content[0].text
    finally:
        stop_event.set()  # Signal the heartbeat to stop
        heartbeat_thread.join()  # Wait for the heartbeat thread to finish
    if assistant_prompt is not None:
        messages=messages[:-1]
    messages.append({"role": response.role, "content": response_text})
    return response, messages


def splice_strings(s1, s2):
    # Consider only the last 50 characters of s1 for potential overlap
    overlap_check = s1[-50:].rstrip()
    # Trim leading spaces from s2
    s2 = s2.lstrip()
    # Start with the entire overlap_check string
    for i in range(len(overlap_check)):
        # Check if string2 starts with the substring of overlap_check
        if s2.startswith(overlap_check[i:]):
            # We found an overlap. Now snip it out:
            print("An overlap between original and continuation string was found", overlap_check[i:])
            s1 = s1[:-50+i]
    return s1 + s2

def generate_module_gpt(prompt, assistant=None, thread=None):
    global gpt_max_output_tokens, continuation_prompt, debug
    module_text, assistant, thread = message_gpt(prompt, assistant, thread)
    module_text = module_text.removeprefix("```json")
    module_text = repair_json(module_text)
    if is_valid_json(module_text):
        print("Valid JSON returned")
        return module_text, [assistant, thread]
    else:
        print("Invalid JSON returned")
        raise Exception("Invalid JSON returned, repair attempts failed.\n"+module_text)

def generate_module_gemini(prompt, chat_session=None):
    global gemini_max_output_tokens, continuation_prompt, debug
    module, chat_session = message_gemini(prompt, chat_session)
    module_text = module.text
    current_tokens = module.usage_metadata.candidates_token_count
    # validate the JSON. If invalid AND near the maximum output token count, then the output was probably cut off before completion
    while not is_valid_json(module_text) and current_tokens > gemini_max_output_tokens - 50:
        print("Output incomplete; attempting continuation")
        continuation, chat_session = message_gemini(continuation_prompt, chat_session)
        if debug:
            print("Continuation: ", continuation.text)
        current_tokens = continuation.usage_metadata.candidates_token_count
        module_text = splice_strings(module_text, continuation.text)
        if debug:
            print("Spliced: ", module_text)
        # did we finally get valid JSON?
    module_text = repair_json(module_text)
    if is_valid_json(module_text):
        print("Valid JSON returned")
        return module_text, chat_session
    else:
        print("Invalid JSON returned")
        if module_text.rstrip().endswith('"gmf_version": 1}') or module_text.rstrip().endswith('"gmf_version": 2}'):  # try a little trick and clip the end off.
            print("Trimming the gmf_version from the end of string...")
            module_text = module_text[:-19]
            if is_valid_json(module_text):
                print("Valid JSON obtained.")
                return module_text, chat_session
            else:
                print("Trimming did not result in valid JSON")
                raise Exception("Invalid JSON returned, repair attempts failed.\n"+module_text)
        else:
            print("String did not end with gmf_version")
            raise Exception("Invalid JSON returned, repair attempts failed.\n"+module_text)
                                
def generate_module_claude(user_prompt, messages=None):
    global debug, continuation_prompt, debug
    module, messages = message_claude(user_prompt, assistant_prompt=begin_module_output, messages=messages)
    module_text = begin_module_output + module.content[0].text
    stop_reason = module.stop_reason
    while stop_reason == "max_tokens":
        if debug:
            print("Output not complete; attempting continuation")
            print_claude_messages(messages)
        # split the string towards the end to get a smooth continuation
        messages = messages[:-1]
        continuation_jumpstart = module_text[-50:]
        module_text = module_text[:-50]
        messages.append({"role": "assistant","content": module_text})
        rate_limit_sleep()
        continuation, messages = message_claude(continuation_prompt, continuation_jumpstart, messages=messages)
        continuation_text = continuation_jumpstart + continuation.content[0].text
        if debug:
            print_claude_messages(messages)
            print("Continuation: ", continuation_text)
        stop_reason = continuation.stop_reason
        module_text = splice_strings(module_text, continuation_text)
        if debug:
            print("Spliced: ", module_text)
    module_text = repair_json(module_text)
    if is_valid_json(module_text):
        print("Valid JSON returned")
        return module_text, messages
    else:
        print("Invalid JSON returned")
        raise Exception("Invalid JSON returned, repair attempts failed.\n"+module_text)
        
def generate_module(user_prompt, chat_session=None):
    global generator
    if generator=="gemini":
        module, chat_session = generate_module_gemini(user_prompt)
    elif generator=="claude":
        module, chat_session = generate_module_claude(user_prompt)
    elif generator=="gpt":
        module, chat_session = generate_module_gpt(user_prompt)
    else:
        raise Exception("Unspecified or unsupported generator")
    return module, chat_session

In [526]:
# create initial module

## Functions for Parsing the Requirement Number Field

In [527]:
# find out what requirements have or have not been addressed in the module by parsing out the "requirement_number" field
# In the disease profile, the numbering of requirements must be integers, but do not have to be consecutive or in order
# a normal return includes parallel lists, the first containing the requirement numbers, and the second containing the requirements
def ingest_requirements(disease_profile_file):
    with open(disease_profile_file, 'r') as file:
        content = file.readlines()
    numbers = []
    items = []
    for line in content:
        match = re.match(r'^(\d+)\.\s*(.*)', line.strip())
        if match:
            number = int(match.group(1))
            text = match.group(2)
            numbers.append(number)
            items.append(text)
    return numbers, items

# use to create a list of implemented requirements by parsing the "requirement_number" field 
# result is returned as a list of requirement numbers in ascending order, no duplicates
def requirements_addressed(module_text):
    global disease_profile_file
    def traverse(obj):
        if isinstance(obj, dict):
            if "requirement_number" in obj:
                if isinstance(obj["requirement_number"], str):
                    text = obj["requirement_number"]
                    numbers.extend(map(int, re.findall(r'\b\d+\b', text)))
                elif isinstance(obj["requirement_number"], list):
                    for remark in obj["requirement_number"]:
                        if isinstance(remark, str):
                            numbers.extend(map(int, re.findall(r'\b\d+\b', remark)))
            for value in obj.values():
                traverse(value)
        elif isinstance(obj, list):
            for item in obj:
                traverse(item)
    all_requirement_numbers, all_requirements = ingest_requirements(disease_profile_file)
    numbers = []
    json_data = json.loads(module_text)
    traverse(json_data)
    addressed_requirement_numbers = sorted(set(numbers))
    return addressed_requirement_numbers, all_requirement_numbers, all_requirements

def requirements_not_addressed(module_text):
    addressed_requirement_numbers, all_requirement_numbers, all_requirements = requirements_addressed(module_text)
    missing_requirements = sorted(set(all_requirement_numbers) - set(addressed_requirement_numbers))
    return missing_requirements, all_requirement_numbers, all_requirements

In [ ]:
# usage
missing_requirements, all_requirement_numbers, all_requirements = requirements_not_addressed(module_text_1)
print("Initial run missed these requirements: ", missing_requirements)

## Functions for Reviewing a Module

### Import Structural Syntax Validation Script & Submodules List

In [578]:
import csv
import structure_syntax_validator
import importlib

submodules_list = []  # Add your submodules list here

# Read the CSV file
with open('submodules.csv', mode='r') as file:
    reader = csv.reader(file)
    for row in reader:
        submodules_list.append(row[0])  # Assuming each row contains a single column

In [579]:
def review_module_gemini(module_text):
    global background, synthea_reference, synthea_examples, disease_profile, review_prompt, gemini_version
    print("Warning: Given the same module, Gemini review scores can vary by 20 percentage points or more, e.g., between 60% and 80%. We recommend using Claude for reviewing.")
    review_model = gemini.GenerativeModel(model_name=gemini_version, generation_config={"temperature": temp, "max_output_tokens": gemini_max_output_tokens})
    review_session = review_model.start_chat()
    prompt = background + synthea_reference + synthea_examples + disease_profile + review_prompt + "\n<synthea_module> "+ module_text+"\n</synthea_module>"
    review, review_session = message_gemini(prompt, review_session)
    return review.text

def review_module_claude(module_text):
    global background, synthea_reference, synthea_examples, disease_profile, review_prompt, message_claude
    prompt = background + synthea_reference + synthea_examples + disease_profile + review_prompt + "\n<synthea_module> "+ module_text+"\n</synthea_module>"
    review, review_session = message_claude(prompt)
    return review.content[0].text

def review_module_gpt(module_text):
    global background, synthea_reference, synthea_examples, disease_profile, review_prompt, message_claude
    prompt = background + synthea_reference + synthea_examples + disease_profile + review_prompt + "\n<synthea_module> "+ module_text+"\n</synthea_module>"
    review, assistant, thread = message_gpt(prompt)
    return review

def review_module(module_text):
    if reviewer=="gemini":
        review = review_module_gemini(module_text)
    elif reviewer=="claude":
        review = review_module_claude(module_text)
    elif reviewer=="gpt":
        review = review_module_gpt(module_text)
    else:
        raise Exception("Unspecified or unsupported reviewer")
    return review

def review_markdown_to_dataframe(table_content):
    # Split the table into rows
    rows = re.split(r'\n\s*\|', table_content)
    # Replace headers
    headers = ['requirement_number', 'Requirement', 'Explanation', 'Transitions', 'Change', 'Score']
    data = []
    # Process each content row (skip header and separator)
    for row in rows[2:]:
        # Split the row into columns
        columns = [col.strip() for col in re.split(r'\s*\|\s*', row.strip()) if col]
        if len(columns) == len(headers):
            data.append(columns)
    # Create DataFrame
    df = pd.DataFrame(data, columns=headers)
    return df

def calc_summary_score(review_text):
# Extract the table content
    table_start = review_text.find('|')
    table_end = review_text.rfind('|')
    if table_start==-1 or table_end==-1 or table_start==table_end:
        # review_text does not have a table
        return None, None
    table_content = review_text[table_start:table_end+1].strip()
    review_df = review_markdown_to_dataframe(table_content)
    # Occasionally, the score column contains a text, such as "N/A". safe_float_convert replaces non-convertable entries with 0.0.
    individual_scores = review_df['Score'].apply(safe_float_convert)
    review_df['Score'] = individual_scores
    # sum up the Score column and divide by the number of entries
    # This can fail when len(review_df) == 0
    if len(review_df) > 0:
        overall_score = 100*(individual_scores.sum())/len(review_df)
    else:
        overall_score = None
    return review_df, overall_score

def safe_float_convert(x):
    try:
        return float(x)
    except ValueError:
        return 0.0
    
def select_lowest_n_scores(df, score_column, n):
    df_filtered = df[df[score_column] < 1.0]
    # If the number of remaining rows is less than or equal to n, return all rows in random order
    if len(df_filtered) <= n:
        return df_filtered.sample(frac=1, random_state=np.random.randint(0, 10000))
    df_sorted = df_filtered.sort_values(by=score_column)
    cumulative_counts = df_sorted[score_column].value_counts().sort_index().cumsum()
    cutoff_score = cumulative_counts[cumulative_counts >= n].index[0]
    result = df_sorted[df_sorted[score_column] <= cutoff_score]
    # If we have more rows than needed, randomly select the required number
    if len(result) > n:
        lower_scores = result[result[score_column] < cutoff_score]
        needed_from_cutoff = n - len(lower_scores)
        cutoff_rows = result[result[score_column] == cutoff_score]
        selected_cutoff_rows = cutoff_rows.sample(n=needed_from_cutoff, random_state=np.random.randint(0, 10000))
        result = pd.concat([lower_scores, selected_cutoff_rows])
    # Randomly shuffle the result
    result = result.sample(frac=1, random_state=np.random.randint(0, 10000))
    return result

def select_low_scoring_requirements(review_df, n):
    low_review_df = select_lowest_n_scores(review_df, 'Score', n)
    # Randomize the order of rows to reduce the bias towards requirements that appear early versus late
    low_review_df = low_review_df.sample(frac=1).reset_index(drop=True)
    failed_requirements = []
    failed_requirement_numbers = []
    for i, row in low_review_df.iterrows():
        requirement = "\nRequirement " + row['requirement_number'] + " has not been satisfactorily implemented (score = "+str(row['Score'])+"). "+row['Explanation']+row['Transitions']+"\nRECOMMENDATION: "+row['Change']
        failed_requirements.append(requirement)
        failed_requirement_numbers.append(row['requirement_number'])
    # to what degree does the review agree with requirements that do not appear in the explanation (not_addressed)?
    return failed_requirements, failed_requirement_numbers

def improve_module(module_text, review_df, n):
    global background, synthea_reference, synthea_examples, update_prompt
    failed_requirements, failed_requirement_numbers = select_low_scoring_requirements(review_df, n)
    print("The following are the lowest rated ", n, " requirements: ", failed_requirement_numbers)
    print("Generating revised module...")
    #revised_module_text, messages = generate_module(update_prompt.format(failed_requirements, module_text))  # QUESTION: Should the disease profile, reference, and examples be included here? (this is a fresh chat with no history)
    revised_module_text, messages = generate_module(background + synthea_reference + synthea_examples + update_prompt.format(failed_requirements, module_text)) 
    return revised_module_text

def review_and_score_module(module_text):
    print("Reviewing...")
    review_text = review_module(module_text)
    #print(review_text)
    review_df, overall_score = calc_summary_score(review_text)
    return review_text, review_df, overall_score

def rate_limit_sleep(seconds=RATE_LIMIT_SLEEP):
    if(generator == "claude" and reviewer == "claude" and seconds>0): 
        print(f"Sleeping for {seconds} seconds to avoid rate limit...")
        time.sleep(seconds)
    
def run_stucture_syntax_checks(module_text):
    global submodules_list
    errors, warnings = structure_syntax_validator.run_all_checks(module_text, submodules_list)
    errors = [e for e in errors if "attribute" not in e.lower()]
    warnings = [w for w in warnings if "attribute" not in w.lower()]
    warnings = [w for w in warnings if "requirement_number" not in w.lower()] 
    # MK - change to errors only (too many warnings)
    #    if errors or warnings:
#        issues = "\n".join([f"Error {i+1}: {e}" for i, e in enumerate(errors)] + [f"Warning {i+1}: {w}" for i, w in enumerate(warnings)])
    if errors:
        issues = ("\n".join([f"Error {i+1}: {e}" for i, e in enumerate(errors)])).strip()
    else:
        issues = None
    return errors, warnings, issues

def detect_and_fix_level_1_problems(module_text, max_attempts = 1):
    errors, warnings, issues = run_stucture_syntax_checks(module_text)
    if issues is None or issues == "":
        # Level 1 validation passed
        return None, module_text
    else:
        attempt = 1
        while issues is not None and issues != "" and attempt <= max_attempts:  
            print("Attempt #", attempt, "to fix Level 1 issues:\n", issues)
            #print(module_text)
            prompt = structure_syntax_improve_prompt.format(module_text, issues)
            prompt = background + synthea_reference + synthea_examples + disease_profile + prompt
            module_text, chat_session = generate_module(prompt)
            errors, warnings, issues = run_stucture_syntax_checks(module_text)
            rate_limit_sleep()
            attempt+=1
        return issues, module_text

def iterative_module_improvement(initial_module_text=None, n_iterations = 10, n_requirements_to_address = 10, generator='claude', reviewer='claude', results=None):
    if initial_module_text is None and results is None:
        raise Exception("Either initial_module_text or previous results must be provided")     
    if results is not None:  #continuation
        module_text = results[-1]['module_text']
        review_df = results[-1]['review_df']
        is_continuation = True
    else:   #no review yet
        module_text = initial_module_text
        results = []
        is_continuation = False
    # Main Iteration
    for i in range(1, n_iterations + 1):
        rate_limit_sleep()
        # If there is an initial module, it needs to be reviewed, so we are going to skip generation step and go directly to the review.   
        if is_continuation:
            try:
                module_text = improve_module(module_text, review_df, n_requirements_to_address)
            except Exception as e: 
                print(e)
                return results
        # Level 1 validation
        issues, module_text = detect_and_fix_level_1_problems(module_text)
        if issues is None or issues =="":
            print("Passed Level 1 validation");
        else:
            print("Level 1 remaining issues: \n", issues)
        # Level 2 validation
        review_text, review_df, overall_score = review_and_score_module(module_text)
        if review_df is None:
            print("Review unsuccessful, terminating after ", i - 1, " iterations.")
            print("Review text: ", review_text)
            return results
        results.append({
            'iteration': i,
            'module_text': module_text,
            'review_text': review_text,
            'review_df': review_df,
            'overall_score': overall_score
        })
        is_continuation = True
        print(f"Overall score for module_text_{i} = {round(overall_score, 1)}")       
        # Sleep if using Claude for both generation and review
        if i < n_iterations: rate_limit_sleep() 
    return results

def count_states(module_text):
    data = json.loads(module_text)
    return len(data['states'])

# functions to save the data for later analysis
def serialize_data(results, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    serialized_results = []
    for i, result in enumerate(results):
        serialized_result = {
            'iteration': result['iteration'],
            'module_text': result['module_text'],  # Already JSON string
            'review_text': result['review_text'],
            'overall_score': result['overall_score'],
            'review_df': result['review_df'].to_dict(orient='records')
        }
        serialized_results.append(serialized_result)
    # Save the entire structure as a single JSON file
    with open(os.path.join(output_dir, 'results.json'), 'w') as f:
        json.dump(serialized_results, f, indent=2)

def deserialize_data(input_dir):
    with open(os.path.join(input_dir, 'results.json'), 'r') as f:
        results = json.load(f)
    for result in results:
        result['review_df'] = pd.DataFrame.from_records(result['review_df'])
    return results


### Revise the Module Based on Review

In [580]:
results = deserialize_data('HT_0829_claude_progressive_1_partial')
len(results)

5

In [581]:
# redo the loop so it returns results at each iteration
importlib.reload(structure_syntax_validator)  # in case of editing-on-the-fly
#module_text = """
#{"name": "Hyperthyroidism", "remarks": "Models the progression of hyperthyroidism with three different etiologies (Graves disease, toxic nodular goiter, and thyroiditis).", "states": {"Initial": {"type": "Initial", "direct_transition": "Delay_Until_Hyperthyroidism", "requirement_number": "1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14", "remarks": "Distributes patients into different hyperthyroidism cohorts based on age, gender, and presence of hyperthyroidism risk factors. Each cohort has a different probability of developing hyperthyroidism."}, "Delay_Until_Hyperthyroidism": {"type": "Delay", "range": {"low": 15, "high": 80, "unit": "years"}, "distributed_transition": [{"distribution": {"attribute": "hyperthyroidism_risk", "default": 0.0195}, "transition": "Hyperthyroidism"}, {"distribution": {"attribute": "subclinical_hyperthyroidism_risk", "default": 0.0273}, "transition": "Subclinical_Hyperthyroidism"}, {"distribution": {"attribute": "no_hyperthyroidism_risk", "default": 0.9532}, "transition": "Terminal"}], "requirement_number": "1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14", "remarks": "Delays until the patient reaches an age at which they may develop hyperthyroidism. Then, based on the calculated probability for the patient's age and gender cohort, the patient transitions to one of three states: Hyperthyroidism, Subclinical_Hyperthyroidism, or Terminal. Note that the default probabilities are for women aged 15-60 without any hyperthyroidism risk factors."}, "Hyperthyroidism": {"type": "Simple", "distributed_transition": [{"distribution": 0.75, "transition": "Graves_Disease"}, {"distribution": 0.175, "transition": "Toxic_Nodular_Goiter"}, {"distribution": 0.075, "transition": "Thyroiditis"}], "requirement_number": "15", "remarks": "Distributes hyperthyroidism patients into one of three etiologies: Graves disease, toxic nodular goiter, or thyroiditis, using the relative incidence rates for the 15-60 age group. Patients over age 60 will have a different distribution of etiologies, which will be modeled later in the module."}, "Graves_Disease": {"type": "ConditionOnset", "assign_to_attribute": "hyperthyroidism_cause", "codes": [{"system": "SNOMED-CT", "code": "35665009", "display": "Graves' disease"}], "direct_transition": "Hyperthyroidism_Symptoms", "requirement_number": "21", "remarks": "Graves disease is an autoimmune disease that leads to overproduction of thyroid hormone. Assigns 'hyperthyroidism_cause' attribute to 'Graves_Disease', then transitions to Hyperthyroidism_Symptoms."}, "Toxic_Nodular_Goiter": {"type": "Simple", "distributed_transition": [{"distribution": 0.8, "transition": "Toxic_Nodular_Goiter_Onset"}, {"distribution": 0.2, "transition": "Severe_Toxic_Nodular_Goiter_Onset"}], "requirement_number": "15", "remarks": "Distributes toxic nodular goiter patients into mild/moderate (80%) or severe (20%) based on disease profile. Severe cases are those with large, suspicious, or malignant thyroid nodules."}, "Toxic_Nodular_Goiter_Onset": {"type": "ConditionOnset", "assign_to_attribute": "hyperthyroidism_cause", "codes": [{"system": "SNOMED-CT", "code": "111591003", "display": "Toxic multinodular goiter"}], "direct_transition": "Hyperthyroidism_Symptoms", "requirement_number": "15", "remarks": "Onsets toxic nodular goiter (mild or moderate severity). Assigns 'hyperthyroidism_cause' attribute to 'Toxic_Nodular_Goiter_Onset', then transitions to Hyperthyroidism_Symptoms."}, "Severe_Toxic_Nodular_Goiter_Onset": {"type": "ConditionOnset", "assign_to_attribute": "hyperthyroidism_cause", "codes": [{"system": "SNOMED-CT", "code": "111591003", "display": "Toxic multinodular goiter"}], "direct_transition": "Hyperthyroidism_Symptoms", "requirement_number": "15", "remarks": "Onsets toxic nodular goiter (severe severity). Assigns 'hyperthyroidism_cause' attribute to 'Severe_Toxic_Nodular_Goiter_Onset', then transitions to Hyperthyroidism_Symptoms."}, "Thyroiditis": {"type": "ConditionOnset", "assign_to_attribute": "hyperthyroidism_cause", "codes": [{"system": "SNOMED-CT", "code": "77586002", "display": "Thyroiditis"}], "direct_transition": "Hyperthyroidism_Symptoms", "requirement_number": "32", "remarks": "Thyroiditis is inflammation of the thyroid gland. Assigns 'hyperthyroidism_cause' attribute to 'Thyroiditis', then transitions to Hyperthyroidism_Symptoms."}, "Subclinical_Hyperthyroidism": {"type": "Simple", "distributed_transition": [{"distribution": 0.75, "transition": "Subclinical_Graves_Disease"}, {"distribution": 0.175, "transition": "Subclinical_Toxic_Nodular_Goiter"}, {"distribution": 0.075, "transition": "Subclinical_Thyroiditis"}], "requirement_number": "30", "remarks": "Distributes subclinical hyperthyroidism patients into one of three etiologies: Graves disease, toxic nodular goiter, or thyroiditis, using the relative incidence rates for the 15-60 age group. Patients over age 60 will have a different distribution of etiologies, which will be modeled later in the module."}, "Subclinical_Graves_Disease": {"type": "ConditionOnset", "assign_to_attribute": "hyperthyroidism_cause", "codes": [{"system": "SNOMED-CT", "code": "35665009", "display": "Graves' disease"}], "direct_transition": "Subclinical_Hyperthyroidism_Monitoring", "requirement_number": "30", "remarks": "Graves disease is an autoimmune disease that leads to overproduction of thyroid hormone. Assigns 'hyperthyroidism_cause' attribute to 'Graves_Disease', then transitions to Subclinical_Hyperthyroidism_Monitoring."}, "Subclinical_Toxic_Nodular_Goiter": {"type": "ConditionOnset", "assign_to_attribute": "hyperthyroidism_cause", "codes": [{"system": "SNOMED-CT", "code": "111591003", "display": "Toxic multinodular goiter"}], "direct_transition": "Subclinical_Hyperthyroidism_Monitoring", "requirement_number": "30", "remarks": "Onsets toxic nodular goiter. Assigns 'hyperthyroidism_cause' attribute to 'Toxic_Nodular_Goiter', then transitions to Subclinical_Hyperthyroidism_Monitoring."}, "Subclinical_Thyroiditis": {"type": "ConditionOnset", "assign_to_attribute": "hyperthyroidism_cause", "codes": [{"system": "SNOMED-CT", "code": "77586002", "display": "Thyroiditis"}], "direct_transition": "Subclinical_Hyperthyroidism_Monitoring", "requirement_number": "30", "remarks": "Thyroiditis is inflammation of the thyroid gland. Assigns 'hyperthyroidism_cause' attribute to 'Thyroiditis', then transitions to Subclinical_Hyperthyroidism_Monitoring."}, "Hyperthyroidism_Symptoms": {"type": "Simple", "distributed_transition": [{"distribution": 0.85, "transition": "Hyperthyroidism_Symptom_1"}, {"distribution": 0.15, "transition": "Uncontrolled_Hyperthyroidism"}], "requirement_number": "41", "remarks": "Distributes patients into those presenting for medical care (85%) and those who will go untreated (15%)."}, "Hyperthyroidism_Symptom_1": {"type": "Symptom", "symptom": "Unexplained Weight Loss", "probability": 0.7, "range": {"low": 20, "high": 60}, "direct_transition": "Hyperthyroidism_Symptom_2", "requirement_number": "15", "remarks": "Unexplained weight loss is a common symptom of hyperthyroidism, affecting 70% of patients. Severity is in the range of 20-60%."}, "Hyperthyroidism_Symptom_2": {"type": "Symptom", "symptom": "Heart Palpitations", "probability": 0.8, "range": {"low": 20, "high": 70}, "direct_transition": "Hyperthyroidism_Symptom_3", "requirement_number": "16", "remarks": "Heart palpitations are a common symptom of hyperthyroidism, affecting 80% of patients. Severity is in the range of 20-70%."}, "Hyperthyroidism_Symptom_3": {"type": "Symptom", "symptom": "Insomnia", "probability": 0.7, "range": {"low": 20, "high": 80}, "direct_transition": "Hyperthyroidism_Symptom_4", "requirement_number": "17", "remarks": "Insomnia is a common symptom of hyperthyroidism, affecting 70% of patients. Severity is in the range of 20-80%."}, "Hyperthyroidism_Symptom_4": {"type": "Symptom", "symptom": "Heat Intolerance", "probability": 0.6, "range": {"low": 20, "high": 50}, "direct_transition": "Hyperthyroidism_Symptom_5", "requirement_number": "18", "remarks": "Heat intolerance is a common symptom of hyperthyroidism, affecting 60% of patients. Severity is in the range of 20-50%."}, "Hyperthyroidism_Symptom_5": {"type": "Symptom", "symptom": "Tremor", "probability": 0.5, "range": {"low": 20, "high": 80}, "direct_transition": "Hyperthyroidism_Symptom_6", "requirement_number": "19", "remarks": "Tremor is a common symptom of hyperthyroidism, affecting 50% of patients. Severity is in the range of 20-80%."}, "Hyperthyroidism_Symptom_6": {"type": "Symptom", "symptom": "Hyperdefecation", "probability": 0.25, "range": {"low": 20, "high": 50}, "direct_transition": "Hyperthyroidism_Symptom_7", "requirement_number": "20", "remarks": "Hyperdefecation is a less common symptom of hyperthyroidism, affecting 25% of patients. Severity is in the range of 20-50%."}, "Hyperthyroidism_Symptom_7": {"type": "Symptom", "symptom": "Ophthalmopathy", "probability": 0.33, "range": {"low": 20, "high": 100}, "direct_transition": "Delay_Until_Diagnosis", "requirement_number": "21", "remarks": "Ophthalmopathy (eye problems) is a symptom primarily of Graves disease, affecting 33% of patients with Graves disease. Severity is in the range of 20-100%."}, "Delay_Until_Diagnosis": {"type": "Delay", "range": {"low": 1, "high": 3, "unit": "weeks"}, "direct_transition": "Initial_Encounter", "requirement_number": "22", "remarks": "Delays for 1-3 weeks to allow symptoms to develop before the patient seeks medical care. Then transitions to Initial_Encounter."}, "Initial_Encounter": {"type": "Encounter", "encounter_class": "ambulatory", "reason": "Heart Palpitations", "codes": [{"system": "SNOMED-CT", "code": "185345009", "display": "Encounter for symptom"}], "direct_transition": "Order_T3_and_FT4", "requirement_number": "22", "remarks": "Initial encounter for hyperthyroidism symptoms, most likely heart palpitations. Transitions to Order_T3_and_FT4."}, "Order_T3_and_FT4": {"type": "Simple", "conditional_transition": [{"condition": {"condition_type": "Attribute", "attribute": "hyperthyroidism_cause", "operator": "==", "value": "Thyroiditis"}, "transition": "Order_TSH_Thyroiditis"}, {"condition": {"condition_type": "Attribute", "attribute": "subclinical_hyperthyroidism_risk", "operator": "is not nil"}, "transition": "Normal_T3_and_FT4"}, {"transition": "Elevated_T3_or_FT4"}], "requirement_number": "22, 30, 32", "remarks": "Orders T3 and FT4 tests. For patients with thyroiditis, the module transitions to Order_TSH_Thyroiditis to reflect the fact that thyroiditis will resolve on its own, and no other testing or treatment is necessary. For patients with subclinical hyperthyroidism, transitions to Normal_T3_and_FT4. Otherwise, transitions to Elevated_T3_or_FT4."}, "Elevated_T3_or_FT4": {"type": "Observation", "category": "laboratory", "codes": [{"system": "LOINC", "code": "1988-5", "display": "T3"}, {"system": "LOINC", "code": "3024-7", "display": "FT4"}], "value_code": {"system": "SNOMED-CT", "code": "278644008", "display": "High"}, "direct_transition": "Order_TSH", "requirement_number": "22", "remarks": "Records elevated T3 and/or FT4. Transitions to Order_TSH."}, "Normal_T3_and_FT4": {"type": "Observation", "category": "laboratory", "codes": [{"system": "LOINC", "code": "1988-5", "display": "T3"}, {"system": "LOINC", "code": "3024-7", "display": "FT4"}], "value_code": {"system": "SNOMED-CT", "code": "177447007", "display": "Normal"}, "direct_transition": "Order_TSH_Subclinical", "requirement_number": "22", "remarks": "Records normal T3 and FT4. Transitions to Order_TSH_Subclinical."}, "Order_TSH": {"type": "Observation", "category": "laboratory", "codes": [{"system": "LOINC", "code": "3948-4", "display": "TSH"}], "value_code": {"system": "SNOMED-CT", "code": "184330003", "display": "Low"}, "direct_transition": "Assess_For_Goiter", "requirement_number": "22", "remarks": "Records low TSH. Transitions to Assess_For_Goiter."}, "Order_TSH_Subclinical": {"type": "Observation", "category": "laboratory", "codes": [{"system": "LOINC", "code": "3948-4", "display": "TSH"}], "value_code": {"system": "SNOMED-CT", "code": "184330003", "display": "Low"}, "direct_transition": "End_Initial_Encounter_Subclinical", "requirement_number": "30", "remarks": "Records low TSH. Transitions to End_Initial_Encounter_Subclinical."}, "Order_TSH_Thyroiditis": {"type": "Observation", "category": "laboratory", "codes": [{"system": "LOINC", "code": "3948-4", "display": "TSH"}], "value_code": {"system": "SNOMED-CT", "code": "184330003", "display": "Low"}, "direct_transition": "End_Initial_Encounter_Thyroiditis", "requirement_number": "32", "remarks": "Records low TSH. Transitions to End_Initial_Encounter_Thyroiditis."}, "Assess_For_Goiter": {"type": "Simple", "conditional_transition": [{"condition": {"condition_type": "Or", "conditions": [{"condition_type": "Symptom", "symptom": "Ophthalmopathy", "operator": ">", "value": 0}, {"condition_type": "Attribute", "attribute": "goiter", "operator": "==", "value": "present"}]}, "transition": "Order_TRAbs"}, {"transition": "Order_RAIU"}], "requirement_number": "23, 24", "remarks": "If the patient has ophthalmopathy or a palpable goiter, transitions to Order_TRAbs. Otherwise, transitions to Order_RAIU."}, "Order_TRAbs": {"type": "Observation", "category": "laboratory", "codes": [{"system": "LOINC", "code": "49213-4", "display": "TRAbs"}], "direct_transition": "Evaluate_TRAbs", "requirement_number": "23", "remarks": "Orders TRAbs. Transitions to Evaluate_TRAbs."}, "Evaluate_TRAbs": {"type": "Simple", "conditional_transition": [{"condition": {"condition_type": "Observation", "codes": [{"system": "LOINC", "code": "49213-4", "display": "TRAbs"}], "operator": "==", "value_code": {"system": "SNOMED-CT", "code": "278644008", "display": "High"}}, "transition": "Graves_Disease_Diagnosis"}, {"transition": "Order_RAIU"}], "requirement_number": "25, 26", "remarks": "Evaluates TRAbs results. If elevated, transitions to Graves_Disease_Diagnosis. Otherwise, transitions to Order_RAIU."}, "Graves_Disease_Diagnosis": {"type": "ConditionOnset", "codes": [{"system": "SNOMED-CT", "code": "35665009", "display": "Graves' disease"}], "direct_transition": "End_Initial_Encounter", "requirement_number": "24, 25", "remarks": "Confirms the diagnosis of Graves disease. Transitions to End_Initial_Encounter."}, "Order_RAIU": {"type": "Simple", "conditional_transition": [{"condition": {"condition_type": "Or", "conditions": [{"condition_type": "Attribute", "attribute": "pregnant", "operator": "==", "value": true}, {"condition_type": "Attribute", "attribute": "lactating", "operator": "==", "value": true}]}, "transition": "Thyroid_Ultrasound"}, {"transition": "Radioactive_Iodine_Uptake"}], "requirement_number": "26", "remarks": "Orders RAIU. If the patient is pregnant or lactating, transitions to Thyroid_Ultrasound because RAIU is contraindicated. Otherwise, transitions to Radioactive_Iodine_Uptake."}, "Radioactive_Iodine_Uptake": {"type": "Observation", "category": "procedure", "codes": [{"system": "LOINC", "code": "17862-6", "display": "RAIU"}], "direct_transition": "Evaluate_RAIU", "requirement_number": "27, 28, 29", "remarks": "Performs and records RAIU. Transitions to Evaluate_RAIU."}, "Thyroid_Ultrasound": {"type": "Procedure", "codes": [{"system": "SNOMED-CT", "code": "428561000124100", "display": "Ultrasound of thyroid"}], "direct_transition": "Evaluate_Ultrasound", "requirement_number": "26", "remarks": "Performs Thyroid Ultrasound. Transitions to Evaluate_Ultrasound."}, "Evaluate_RAIU": {"type": "Simple", "conditional_transition": [{"condition": {"condition_type": "And", "conditions": [{"condition_type": "Attribute", "attribute": "hyperthyroidism_cause", "operator": "==", "value": "Graves_Disease"}, {"condition_type": "Observation", "codes": [{"system": "LOINC", "code": "17862-6", "display": "RAIU"}], "operator": "==", "value_code": {"system": "SNOMED-CT", "code": "441772007", "display": "Diffuse"}}]}, "transition": "Graves_Disease_Diagnosis"}, {"condition": {"condition_type": "And", "conditions": [{"condition_type": "Attribute", "attribute": "hyperthyroidism_cause", "operator": "==", "value": "Toxic_Nodular_Goiter"}, {"condition_type": "Observation", "codes": [{"system": "LOINC", "code": "17862-6", "display": "RAIU"}], "operator": "==", "value_code": {"system": "SNOMED-CT", "code": "260217007", "display": "Focal"}}]}, "transition": "Toxic_Nodular_Goiter_Diagnosis"}, {"condition": {"condition_type": "And", "conditions": [{"condition_type": "Attribute", "attribute": "hyperthyroidism_cause", "operator": "==", "value": "Thyroiditis"}, {"condition_type": "Observation", "codes": [{"system": "LOINC", "code": "17862-6", "display": "RAIU"}], "operator": "==", "value_code": {"system": "SNOMED-CT", "code": "184330003", "display": "Low"}}]}, "transition": "Thyroiditis_Diagnosis"}, {"transition": "End_Initial_Encounter"}], "requirement_number": "27, 28, 29", "remarks": "Evaluates RAIU results. If diffuse uptake and the likely diagnosis is Graves disease, transitions to Graves_Disease_Diagnosis. If focal uptake and the likely diagnosis is toxic nodular goiter, transitions to Toxic_Nodular_Goiter_Diagnosis. If low or absent uptake and the likely diagnosis is thyroiditis, transitions to Thyroiditis_Diagnosis. Otherwise, transitions to End_Initial_Encounter."}, "Evaluate_Ultrasound": {"type": "Simple", "conditional_transition": [{"condition": {"condition_type": "And", "conditions": [{"condition_type": "Attribute", "attribute": "hyperthyroidism_cause", "operator": "==", "value": "Toxic_Nodular_Goiter"}, {"condition_type": "Observation", "codes": [{"system": "SNOMED-CT", "code": "428561000124100", "display": "Ultrasound of thyroid"}], "operator": "==", "value_code": {"system": "SNOMED-CT", "code": "442073007", "display": "Nodular"}}]}, "transition": "Toxic_Nodular_Goiter_Diagnosis"}, {"transition": "End_Initial_Encounter"}], "requirement_number": "28", "remarks": "Evaluates Thyroid Ultrasound results. If nodules are present and the likely diagnosis is toxic nodular goiter, transitions to Toxic_Nodular_Goiter_Diagnosis. Otherwise, transitions to End_Initial_Encounter."}, "Toxic_Nodular_Goiter_Diagnosis": {"type": "ConditionOnset", "codes": [{"system": "SNOMED-CT", "code": "111591003", "display": "Toxic multinodular goiter"}], "direct_transition": "End_Initial_Encounter", "requirement_number": "28", "remarks": "Confirms the diagnosis of toxic nodular goiter. Transitions to End_Initial_Encounter."}, "Thyroiditis_Diagnosis": {"type": "ConditionOnset", "codes": [{"system": "SNOMED-CT", "code": "77586002", "display": "Thyroiditis"}], "direct_transition": "End_Initial_Encounter", "requirement_number": "29", "remarks": "Confirms the diagnosis of thyroiditis. Transitions to End_Initial_Encounter."}, "End_Initial_Encounter": {"type": "EncounterEnd", "direct_transition": "Treatment_Decision", "requirement_number": "31", "remarks": "Ends the initial encounter. Transitions to Treatment_Decision."}, "End_Initial_Encounter_Subclinical": {"type": "EncounterEnd", "direct_transition": "Subclinical_Hyperthyroidism_Monitoring", "requirement_number": "30", "remarks": "Ends the initial encounter. Transitions to Subclinical_Hyperthyroidism_Monitoring."}, "End_Initial_Encounter_Thyroiditis": {"type": "EncounterEnd", "direct_transition": "Thyroiditis_Resolution", "requirement_number": "32", "remarks": "Ends the initial encounter. Transitions to Thyroiditis_Resolution."}, "Treatment_Decision": {"type": "Simple", "conditional_transition": [{"condition": {"condition_type": "Attribute", "attribute": "hyperthyroidism_cause", "operator": "==", "value": "Thyroiditis"}, "transition": "Thyroiditis_Resolution"}, {"condition": {"condition_type": "Or", "conditions": [{"condition_type": "Attribute", "attribute": "hyperthyroidism_cause", "operator": "==", "value": "Severe_Toxic_Nodular_Goiter_Onset"}, {"condition_type": "Symptom", "symptom": "Ophthalmopathy", "operator": ">=", "value": 80}]}, "transition": "Delay_For_Surgery"}, {"transition": "Prescribe_Antithyroid_Drugs"}], "requirement_number": "32, 33, 38", "remarks": "Determines the best course of treatment. If the cause is thyroiditis, transitions to Thyroiditis_Resolution. If the cause is severe toxic nodular goiter or the patient has severe ophthalmopathy, transitions to Delay_For_Surgery. Otherwise, transitions to Prescribe_Antithyroid_Drugs."}, "Thyroiditis_Resolution": {"type": "Delay", "range": {"low": 1, "high": 6, "unit": "months"}, "direct_transition": "End_Thyroiditis", "requirement_number": "32", "remarks": "Delays for 1-6 months to allow thyroiditis to resolve on its own. Transitions to End_Thyroiditis."}, "End_Thyroiditis": {"type": "ConditionEnd", "condition_onset": "Thyroiditis", "direct_transition": "Terminal", "requirement_number": "32", "remarks": "Ends the thyroiditis condition. Transitions to Terminal."}, "Prescribe_Antithyroid_Drugs": {"type": "MedicationOrder", "assign_to_attribute": "antithyroid_drug", "reason": "Graves_Disease", "codes": [{"system": "RxNorm", "code": "1731406", "display": "Methimazole 10 MG Oral Tablet"}], "direct_transition": "Prescribe_Beta_Blocker", "requirement_number": "33", "remarks": "Prescribes methimazole, an antithyroid drug. Transitions to Prescribe_Beta_Blocker."}, "Prescribe_Beta_Blocker": {"type": "MedicationOrder", "reason": "Heart Palpitations", "codes": [{"system": "RxNorm", "code": "308022", "display": "Propranolol Hydrochloride 10 MG Oral Tablet"}], "direct_transition": "Antithyroid_Drug_Monitoring", "probability": 0.75, "requirement_number": "31", "remarks": "Prescribes propranolol, a beta blocker. This occurs in 75% of hyperthyroidism cases."}, "Antithyroid_Drug_Monitoring": {"type": "Delay", "exact": {"quantity": 1, "unit": "months"}, "direct_transition": "Followup_Encounter_1", "requirement_number": "34", "remarks": "Delays for one month for antithyroid drug monitoring. Transitions to Followup_Encounter_1."}, "Followup_Encounter_1": {"type": "Encounter", "encounter_class": "ambulatory", "reason": "Graves_Disease", "codes": [{"system": "SNOMED-CT", "code": "185349003", "display": "Encounter for follow up"}], "direct_transition": "Order_T3_and_FT4_Followup", "requirement_number": "34", "remarks": "First followup encounter to monitor antithyroid drug treatment. Transitions to Order_T3_and_FT4_Followup."}, "Order_T3_and_FT4_Followup": {"type": "Observation", "category": "laboratory", "codes": [{"system": "LOINC", "code": "1988-5", "display": "T3"}, {"system": "LOINC", "code": "3024-7", "display": "FT4"}], "direct_transition": "Evaluate_T3_and_FT4_Followup", "requirement_number": "34", "remarks": "Orders T3 and FT4 tests to monitor antithyroid drug treatment. Transitions to Evaluate_T3_and_FT4_Followup."}, "Evaluate_T3_and_FT4_Followup": {"type": "Simple", "conditional_transition": [{"condition": {"condition_type": "Observation", "codes": [{"system": "LOINC", "code": "1988-5", "display": "T3"}, {"system": "LOINC", "code": "3024-7", "display": "FT4"}], "operator": "==", "value_code": {"system": "SNOMED-CT", "code": "177447007", "display": "Normal"}}, "transition": "End_Followup_Encounter_1"}, {"transition": "Adjust_Antithyroid_Drug_Dose"}], "requirement_number": "34", "remarks": "Evaluates T3 and FT4 results. If normal, transitions to End_Followup_Encounter_1. Otherwise, transitions to Adjust_Antithyroid_Drug_Dose."}, "Adjust_Antithyroid_Drug_Dose": {"type": "Simple", "direct_transition": "Antithyroid_Drug_Monitoring", "requirement_number": "34", "remarks": "Adjusts the antithyroid drug dose. Transitions to Antithyroid_Drug_Monitoring."}, "End_Followup_Encounter_1": {"type": "EncounterEnd", "conditional_transition": [{"condition": {"condition_type": "Counter", "attribute": "antithyroid_drug_monitoring_count", "operator": "<", "value": 3}, "transition": "Antithyroid_Drug_Monitoring"}, {"transition": "Evaluate_Antithyroid_Drug_Treatment"}], "requirement_number": "34", "remarks": "Ends the first followup encounter. If the patient has had fewer than three months of monitoring, transitions to Antithyroid_Drug_Monitoring. Otherwise, transitions to Evaluate_Antithyroid_Drug_Treatment."}, "Evaluate_Antithyroid_Drug_Treatment": {"type": "Delay", "exact": {"quantity": 12, "unit": "months"}, "direct_transition": "Evaluate_Antithyroid_Drug_Remission", "requirement_number": "33", "remarks": "Delays for 12 months to allow a total of 15 months of antithyroid drug treatment. Transitions to Evaluate_Antithyroid_Drug_Remission."}, "Evaluate_Antithyroid_Drug_Remission": {"type": "Simple", "distributed_transition": [{"distribution": 0.45, "transition": "End_Antithyroid_Drug_Treatment"}, {"distribution": 0.55, "transition": "Radioactive_Iodine_Therapy"}], "requirement_number": "33, 36", "remarks": "Evaluates whether the antithyroid drug treatment has achieved remission. If so (45% probability), transitions to End_Antithyroid_Drug_Treatment. Otherwise (55% probability), transitions to Radioactive_Iodine_Therapy."}, "End_Antithyroid_Drug_Treatment": {"type": "MedicationEnd", "referenced_by_attribute": "antithyroid_drug", "direct_transition": "Delay_Until_Relapse_Check", "requirement_number": "33", "remarks": "Discontinues the antithyroid drug. Transitions to Delay_Until_Relapse_Check."}, "Delay_Until_Relapse_Check": {"type": "Delay", "exact": {"quantity": 1, "unit": "years"}, "direct_transition": "Check_For_Relapse", "requirement_number": "35", "remarks": "Delays for one year after discontinuation of antithyroid drug treatment. Transitions to Check_For_Relapse."}, "Check_For_Relapse": {"type": "Simple", "distributed_transition": [{"distribution": 0.4, "transition": "Hyperthyroidism_Relapse"}, {"distribution": 0.6, "transition": "Delay_Until_Second_Relapse_Check"}], "requirement_number": "35", "remarks": "Checks for relapse of hyperthyroidism. The relapse rate is 40% within the first year. If relapse occurs, transitions to Hyperthyroidism_Relapse. Otherwise, transitions to Delay_Until_Second_Relapse_Check."}, "Delay_Until_Second_Relapse_Check": {"type": "Delay", "exact": {"quantity": 4, "unit": "years"}, "direct_transition": "Check_For_Second_Relapse", "requirement_number": "35", "remarks": "Delays for four years after the first relapse check. Transitions to Check_For_Second_Relapse."}, "Check_For_Second_Relapse": {"type": "Simple", "distributed_transition": [{"distribution": 0.6, "transition": "Hyperthyroidism_Relapse"}, {"distribution": 0.4, "transition": "Annual_TSH_Monitoring"}], "requirement_number": "35, 40", "remarks": "Checks for relapse of hyperthyroidism. The relapse rate is 60% within five years. If relapse occurs, transitions to Hyperthyroidism_Relapse. Otherwise, transitions to Annual_TSH_Monitoring."}, "Hyperthyroidism_Relapse": {"type": "ConditionOnset", "assign_to_attribute": "hyperthyroidism_relapse", "codes": [{"system": "SNOMED-CT", "code": "230268004", "display": "Recurrent disease"}], "direct_transition": "Radioactive_Iodine_Therapy", "requirement_number": "36", "remarks": "Onsets relapse of hyperthyroidism. Transitions to Radioactive_Iodine_Therapy."}, "Radioactive_Iodine_Therapy": {"type": "Procedure", "codes": [{"system": "SNOMED-CT", "code": "128335006", "display": "Radioactive iodine therapy"}], "direct_transition": "Delay_For_RAI_Evaluation", "requirement_number": "36", "remarks": "Performs radioactive iodine therapy. Transitions to Delay_For_RAI_Evaluation."}, "Delay_For_RAI_Evaluation": {"type": "Delay", "exact": {"quantity": 6, "unit": "months"}, "direct_transition": "Followup_Encounter_2", "requirement_number": "37", "remarks": "Delays for six months after radioactive iodine therapy to allow for evaluation of treatment effectiveness. Transitions to Followup_Encounter_2."}, "Followup_Encounter_2": {"type": "Encounter", "encounter_class": "ambulatory", "reason": "Graves_Disease", "codes": [{"system": "SNOMED-CT", "code": "185349003", "display": "Encounter for follow up"}], "direct_transition": "Order_T3_and_FT4_Followup_2", "requirement_number": "37", "remarks": "Second followup encounter to monitor radioactive iodine treatment effectiveness. Transitions to Order_T3_and_FT4_Followup_2."}, "Order_T3_and_FT4_Followup_2": {"type": "Observation", "category": "laboratory", "codes": [{"system": "LOINC", "code": "1988-5", "display": "T3"}, {"system": "LOINC", "code": "3024-7", "display": "FT4"}], "direct_transition": "Evaluate_T3_and_FT4_Followup_2", "requirement_number": "37", "remarks": "Orders T3 and FT4 tests to monitor radioactive iodine treatment effectiveness. Transitions to Evaluate_T3_and_FT4_Followup_2."}, "Evaluate_T3_and_FT4_Followup_2": {"type": "Simple", "conditional_transition": [{"condition": {"condition_type": "Observation", "codes": [{"system": "LOINC", "code": "1988-5", "display": "T3"}, {"system": "LOINC", "code": "3024-7", "display": "FT4"}], "operator": "==", "value_code": {"system": "SNOMED-CT", "code": "177447007", "display": "Normal"}}, "transition": "End_Followup_Encounter_2"}, {"transition": "Repeat_RAI_Decision"}], "requirement_number": "37", "remarks": "Evaluates T3 and FT4 results. If normal, transitions to End_Followup_Encounter_2. Otherwise, transitions to Repeat_RAI_Decision."}, "Repeat_RAI_Decision": {"type": "Simple", "distributed_transition": [{"distribution": 0.5, "transition": "Radioactive_Iodine_Therapy"}, {"distribution": 0.5, "transition": "Delay_For_Surgery"}], "requirement_number": "37", "remarks": "If radioactive iodine therapy was not successful, the patient will either repeat RAI or have surgical intervention. Transitions to Radioactive_Iodine_Therapy (50% probability) or Delay_For_Surgery (50% probability)."}, "End_Followup_Encounter_2": {"type": "EncounterEnd", "direct_transition": "Annual_TSH_Monitoring", "requirement_number": "40", "remarks": "Ends the second followup encounter. Transitions to Annual_TSH_Monitoring."}, "Delay_For_Surgery": {"type": "Delay", "range": {"low": 1, "high": 3, "unit": "months"}, "direct_transition": "Surgery_Encounter", "requirement_number": "38", "remarks": "Delays for 1-3 months to allow scheduling of surgery. Transitions to Surgery_Encounter."}, "Surgery_Encounter": {"type": "Encounter", "encounter_class": "inpatient", "reason": "Graves_Disease", "codes": [{"system": "SNOMED-CT", "code": "183493006", "display": "Encounter for surgical operation"}], "direct_transition": "Surgery_Decision", "requirement_number": "38", "remarks": "Encounter for surgical intervention. Transitions to Surgery_Decision."}, "Surgery_Decision": {"type": "Simple", "distributed_transition": [{"distribution": 0.8, "transition": "Thyroidectomy"}, {"distribution": 0.2, "transition": "Radiofrequency_Ablation"}], "requirement_number": "38", "remarks": "Decides which surgery will be performed. 80% of patients will undergo thyroidectomy, and 20% will receive radiofrequency ablation. Transitions to Thyroidectomy (80% probability) or Radiofrequency_Ablation (20% probability)."}, "Thyroidectomy": {"type": "Procedure", "codes": [{"system": "SNOMED-CT", "code": "303961006", "display": "Thyroidectomy"}], "direct_transition": "Evaluate_Thyroidectomy", "requirement_number": "38", "remarks": "Performs thyroidectomy. Transitions to Evaluate_Thyroidectomy."}, "Radiofrequency_Ablation": {"type": "Procedure", "codes": [{"system": "SNOMED-CT", "code": "447339001", "display": "Radiofrequency ablation of thyroid"}], "direct_transition": "Evaluate_RFA", "requirement_number": "38", "remarks": "Performs radiofrequency ablation. Transitions to Evaluate_RFA."}, "Evaluate_Thyroidectomy": {"type": "Simple", "distributed_transition": [{"distribution": 0.95, "transition": "End_Surgery_Encounter"}, {"distribution": 0.05, "transition": "Delay_For_Surgery"}], "requirement_number": "38", "remarks": "Evaluates whether the thyroidectomy was successful. The cure rate is 95%. If successful, transitions to End_Surgery_Encounter. Otherwise, transitions to Delay_For_Surgery for repeat surgery."}, "Evaluate_RFA": {"type": "Simple", "distributed_transition": [{"distribution": 0.75, "transition": "End_Surgery_Encounter"}, {"distribution": 0.25, "transition": "Thyroidectomy"}], "requirement_number": "38", "remarks": "Evaluates whether the radiofrequency ablation was successful. The success rate is 75%. If successful, transitions to End_Surgery_Encounter. Otherwise, transitions to Thyroidectomy."}, "End_Surgery_Encounter": {"type": "EncounterEnd", "direct_transition": "Hypothyroidism_Risk", "requirement_number": "39", "remarks": "Ends the surgery encounter. Transitions to Hypothyroidism_Risk."}, "Hypothyroidism_Risk": {"type": "Simple", "distributed_transition": [{"distribution": 0.85, "transition": "Hypothyroidism_Onset"}, {"distribution": 0.15, "transition": "Annual_TSH_Monitoring"}], "requirement_number": "39, 40", "remarks": "Determines whether the patient develops hypothyroidism following surgery. The risk is 85%. Transitions to Hypothyroidism_Onset (85% probability) or Annual_TSH_Monitoring (15% probability)."}, "Hypothyroidism_Onset": {"type": "ConditionOnset", "codes": [{"system": "SNOMED-CT", "code": "3439004", "display": "Hypothyroidism"}], "direct_transition": "Prescribe_Levothyroxine", "requirement_number": "39", "remarks": "Onsets hypothyroidism. Transitions to Prescribe_Levothyroxine."}, "Prescribe_Levothyroxine": {"type": "MedicationOrder", "reason": "Hypothyroidism", "codes": [{"system": "RxNorm", "code": "856744", "display": "Levothyroxine Sodium 0.025 MG Oral Tablet"}], "direct_transition": "Annual_TSH_Monitoring", "requirement_number": "39", "remarks": "Prescribes levothyroxine for hypothyroidism. Transitions to Annual_TSH_Monitoring."}, "Annual_TSH_Monitoring": {"type": "Delay", "exact": {"quantity": 1, "unit": "years"}, "direct_transition": "TSH_Monitoring_Encounter", "requirement_number": "40", "remarks": "Delays for one year for annual TSH monitoring. Transitions to TSH_Monitoring_Encounter."}, "TSH_Monitoring_Encounter": {"type": "Encounter", "encounter_class": "ambulatory", "reason": "Graves_Disease", "codes": [{"system": "SNOMED-CT", "code": "185349003", "display": "Encounter for follow up"}], "direct_transition": "Order_TSH_Followup", "requirement_number": "40", "remarks": "Annual followup encounter for TSH monitoring. Transitions to Order_TSH_Followup."}, "Order_TSH_Followup": {"type": "Observation", "category": "laboratory", "codes": [{"system": "LOINC", "code": "3948-4", "display": "TSH"}], "direct_transition": "Evaluate_TSH_Followup", "requirement_number": "40", "remarks": "Orders TSH test to monitor for recurrence of hyperthyroidism or development of hypothyroidism. Transitions to Evaluate_TSH_Followup."}, "Evaluate_TSH_Followup": {"type": "Simple", "conditional_transition": [{"condition": {"condition_type": "Observation", "codes": [{"system": "LOINC", "code": "3948-4", "display": "TSH"}], "operator": "==", "value_code": {"system": "SNOMED-CT", "code": "184330003", "display": "Low"}}, "transition": "Hyperthyroidism_Relapse"}, {"condition": {"condition_type": "Observation", "codes": [{"system": "LOINC", "code": "3948-4", "display": "TSH"}], "operator": "==", "value_code": {"system": "SNOMED-CT", "code": "278644008", "display": "High"}}, "transition": "Hypothyroidism_Onset"}, {"transition": "End_TSH_Monitoring_Encounter"}], "requirement_number": "40", "remarks": "Evaluates TSH results. If low, transitions to Hyperthyroidism_Relapse. If high, transitions to Hypothyroidism_Onset. If normal, transitions to End_TSH_Monitoring_Encounter."}, "End_TSH_Monitoring_Encounter": {"type": "EncounterEnd", "direct_transition": "Annual_TSH_Monitoring", "requirement_number": "40", "remarks": "Ends the TSH monitoring encounter. Transitions to Annual_TSH_Monitoring."}, "Subclinical_Hyperthyroidism_Monitoring": {"type": "Delay", "exact": {"quantity": 4, "unit": "months"}, "direct_transition": "Subclinical_Followup_Encounter", "requirement_number": "30", "remarks": "Delays for four months for subclinical hyperthyroidism monitoring. Transitions to Subclinical_Followup_Encounter."}, "Subclinical_Followup_Encounter": {"type": "Encounter", "encounter_class": "ambulatory", "reason": "Subclinical_Graves_Disease", "codes": [{"system": "SNOMED-CT", "code": "185349003", "display": "Encounter for follow up"}], "direct_transition": "Order_T3_and_FT4_Subclinical_Followup", "requirement_number": "30", "remarks": "Followup encounter for subclinical hyperthyroidism monitoring. Transitions to Order_T3_and_FT4_Subclinical_Followup."}, "Order_T3_and_FT4_Subclinical_Followup": {"type": "Observation", "category": "laboratory", "codes": [{"system": "LOINC", "code": "1988-5", "display": "T3"}, {"system": "LOINC", "code": "3024-7", "display": "FT4"}], "direct_transition": "Evaluate_T3_and_FT4_Subclinical_Followup", "requirement_number": "30", "remarks": "Orders T3 and FT4 tests to monitor for progression to overt hyperthyroidism. Transitions to Evaluate_T3_and_FT4_Subclinical_Followup."}, "Evaluate_T3_and_FT4_Subclinical_Followup": {"type": "Simple", "conditional_transition": [{"condition": {"condition_type": "Observation", "codes": [{"system": "LOINC", "code": "1988-5", "display": "T3"}, {"system": "LOINC", "code": "3024-7", "display": "FT4"}], "operator": "==", "value_code": {"system": "SNOMED-CT", "code": "278644008", "display": "High"}}, "transition": "Hyperthyroidism"}, {"transition": "End_Subclinical_Followup_Encounter"}], "requirement_number": "30", "remarks": "Evaluates T3 and FT4 results. If elevated, transitions to Hyperthyroidism. If normal, transitions to End_Subclinical_Followup_Encounter."}, "End_Subclinical_Followup_Encounter": {"type": "EncounterEnd", "distributed_transition": [{"distribution": 0.035, "transition": "Hyperthyroidism"}, {"distribution": 0.965, "transition": "Subclinical_Hyperthyroidism_Monitoring"}], "requirement_number": "30", "remarks": "Ends the subclinical hyperthyroidism followup encounter. 3.5% of patients will progress to overt hyperthyroidism each year. Transitions to Hyperthyroidism (3.5% probability) or Subclinical_Hyperthyroidism_Monitoring (96.5% probability)."}, "Uncontrolled_Hyperthyroidism": {"type": "Simple", "direct_transition": "Long_Term_Complications", "requirement_number": "41", "remarks": "Represents patients with uncontrolled hyperthyroidism. Transitions to Long_Term_Complications."}, "Long_Term_Complications": {"type": "Delay", "range": {"low": 5, "high": 20, "unit": "years"}, "distributed_transition": [{"distribution": 0.25, "transition": "Osteoporosis"}, {"distribution": 0.125, "transition": "Atrial_Fibrillation"}, {"distribution": 0.25, "transition": "Cardiovascular_Death"}, {"distribution": 0.375, "transition": "Terminal"}], "requirement_number": "42, 43, 44", "remarks": "Delays for 5-20 years to allow for the development of long-term complications of uncontrolled hyperthyroidism. 25% of patients will develop osteoporosis, 12.5% will develop atrial fibrillation, and 25% will experience cardiovascular death. Transitions to Osteoporosis (25% probability), Atrial_Fibrillation (12.5% probability), Cardiovascular_Death (25% probability) or Terminal (37.5% probability)."}, "Osteoporosis": {"type": "ConditionOnset", "codes": [{"system": "SNOMED-CT", "code": "64859006", "display": "Osteoporosis"}], "direct_transition": "Terminal", "requirement_number": "42", "remarks": "Onsets osteoporosis. Transitions to Terminal."}, "Atrial_Fibrillation": {"type": "ConditionOnset", "codes": [{"system": "SNOMED-CT", "code": "49436004", "display": "Atrial fibrillation"}], "direct_transition": "Terminal", "requirement_number": "43", "remarks": "Onsets atrial fibrillation. Transitions to Terminal."}, "Cardiovascular_Death": {"type": "Death", "codes": [{"system": "SNOMED-CT", "code": "53741008", "display": "Death due to cardiovascular disease"}], "direct_transition": "Terminal", "requirement_number": "44", "remarks": "Causes death from cardiovascular disease. Transitions to Terminal."}, "Terminal": {"type": "Terminal", "requirement_number": "", "remarks": "Represents the end of the module."}}, "gmf_version": 1}
#"""
module_text = None
#results = None
for i in range(6, 11):
    print("\nITERATION ",i)
    results = iterative_module_improvement(initial_module_text=module_text, n_iterations=1, n_requirements_to_address=12, generator=generator, reviewer=reviewer, results = results)
    module_text = None
    print([element['overall_score'] for element in results])
    print([count_states(element['module_text']) for element in results])


ITERATION  6
Sleeping for 5 seconds to avoid rate limit...
The following are the lowest rated  12  requirements:  ['1', '13']
Generating revised module...
Total time taken: 98 seconds
Sleeping for 5 seconds to avoid rate limit...
Total time taken: 33 seconds
Valid JSON returned
Checking state: Mild_Moderate_Ophthalmopathy
  Checking edge from Mild_Moderate_Ophthalmopathy to Mild_Moderate_Ophthalmopathy
Found first state: Mild_Moderate_Ophthalmopathy
Checking state: Normal_RAIU
  Checking edge from Normal_RAIU to Normal_RAIU
Found first state: Normal_RAIU
Checking state: Elevated_TRAbs
  Checking edge from Elevated_TRAbs to Elevated_TRAbs
Found first state: Elevated_TRAbs
Checking state: Diffuse_RAIU
  Checking edge from Diffuse_RAIU to Diffuse_RAIU
Found first state: Diffuse_RAIU
Checking state: Severe_Ophthalmopathy
  Checking edge from Severe_Ophthalmopathy to Severe_Ophthalmopathy
Found first state: Severe_Ophthalmopathy
Attempt # 1 to fix Level 1 issues:
 Error 1:  The state 'Seve

In [582]:
len(results)

10

In [583]:
serialize_data(results, 'HT_0830_claude_progressive_1')